In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
from moscowhack.backend.postgres import get_postgres_connection, SkolkovoProm

In [3]:
psql_db = get_postgres_connection()

In [6]:
psql_db.bind([SkolkovoProm])

In [8]:
with psql_db:
    raw_select = SkolkovoProm.select()
    records = list(raw_select.dicts())

In [14]:
records[0]

{'name': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "СИБИРСКИЕ ВОДНЫЕ ТЕХНОЛОГИИ"',
 'inn': '5404407249',
 'ogrn': '1105404001900',
 'description': "['ООО «Сибирские водные технологии» - это команда ученых и инженеров, работающих над созданием умных систем водоочистки, способных в режиме реального времени контролировать состав воды и качество ее очистки, корректируя, исходя из реальной кондиции загрязненной воды, очистительную программу. Компания работает над разработкой саморегулируемой экосистемы продуктов, которая позволит создавать автономные объекты, функционирующие в режиме постоянного оборота воды, в которых вода после очистки направлялась бы на повторное использование.', 'Специалистами компании разработан ряд технологий: гетерокоагуляции, контроля состава водных сред в потоке, раздельной обработки сточных вод.', 'В команде проекта 7 человек. Основатель компании и ведущий разработчик - доцент кафедры химии Новосибирского государственного университета, автор более 20-ти патентов п

In [3]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [8]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, max_length=2048, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


In [ ]:
embeds = {
    record['inn']:
    embed_bert_cls(
        record['description'].replace("['", "").replace("']", '')
        model, tokenizer
    )
    for record in records
}
